# SQL for Data Analysis  
## Advanced Workshop
*D‑Lab, UC Berkeley*

<div class="alert alert-success"> 
<b>Prerequisites for SQL Advanced Workshop</b><br><br>
Completion of "SQL for Data Analysis: Introductory Workshop" or equivalent experience:
<ul>
<li>Understanding of basic SQL syntax including SELECT, FROM, WHERE, GROUP BY</li>
<li>Familiarity with basic data filtering and sorting in SQL</li>
<li>Experience with simple aggregations (COUNT, SUM, AVG)</li>
</ul>
    
These prerequisites ensure participants have the foundational knowledge needed to succeed in learning the more advanced concepts like JOINs, subqueries, CTEs, and window functions covered in the second workshop.
</div>

## Table of Contents
1. [Introduction](#introduction)
2. [Relational Joins](#joins)
3. [Subqueries](#subqueries)
4. [Common Table Expressions (CTEs)](#ctes)
5. [Pivoting and Unpivoting](#pivot)
6. [Window Functions](#window)
7. [Key Points](#keypoints)

<div class="alert alert-success"> 
<b>Learning Goals</b><br><br>
By the end of this workshop you will be able to:
<ul>
<li>Combine data from multiple tables using different types of JOINs (INNER, LEFT, SELF).</li>
<li>Understand the role of primary and foreign keys in establishing table relationships.</li>
<li>Write and use subqueries to break down complex queries with multiple logical steps.</li>
<li>Simplify complex queries using Common Table Expressions (CTEs).</li>
<li>Transform data between row and column orientations with pivoting and unpivoting techniques.</li>
<li>Apply window functions to perform calculations across specified sets of rows.</li>
</ul>
</div>

### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>
📝 **Poll:** A Zoom poll to help you learn!<br>
🎬 **Demo**: Showing off something more advanced – so you know what Python can be used for!<br>

In [120]:
# Here are some packages that we will need:

from IPython.display import SVG
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

<a id='introduction'></a>
## 1 · Introduction 
In the last workshop we covered many different operations one can perform in a given table. However, going back to the idea of Relational Databases, we often won't have all of the information we need on a single table, and we will need to find a way of cross-referencing the information from two (or more) different tables together. This processing is called "Joining", and it is an essential aspect of querying with SQL. 

Another common procedure is to first modify the data, and then perform some operations in the modified data. This is what Subqueries are used for - they allow us to quickly reference a modified version of the dataset we are querying, or compare information between two tables without joining them. Common Table Expressions are a way of doing many subqueries simultaneously, while keeping things readable and organized.

Lastly, we will cover a class of operations called Window Functions, which are some of the most powerful tools in SQL. Instead of performing the same function to all rows, they allow the functions to only be applied to *windows*, which are a subset of the observations that are related in a prespecified way. 


<a id='joins'></a>
## 2 · Relational Joins 
**Purpose:** JOINs allow us to combine information from multiple different tables into one. When used with Querying, this also provides a way of retaining only the information required for a particular analysis into a single, organized table, even if originally this information was spread out  

**Example:** Say that we have two tables, one recording the information about Managers, and another one with the information about Employees. Each employee produces a certain amount of revenue to the firm, but a higher-up is interested in understanding what Manager had direct employees that produce the most revenue. 

If we had a table that listed each manager, their respective employees and how much they earned, we could use SUM + GROUP BY to quickly do this analysis. But we don't - the information is stored in two separate tables. This is exactly the context of `JOIN`s - creating a new table combining the information from two (or more) other tables. Here is a quick diagram to understand what is happening:

![Database Relationship Diagram](database-relationship-diagram2.svg)
---

Notice that in this example, employee_id would be the *primary key* of each table - a unique identifier for each row - while the column "report_to" on the Employees table serves as a *foreign key* - it serves as a reference to values on the table Managers which can be used to cross-reference information.

Notice also that we didn't keep all of the information from each table in the end result. By querying the table after joining, we can decide what to retain, and even perform operations or filtering directly!

In [9]:
# 1. Create the dataframes for Managers and Employees
# Managers DataFrame
managers_data = {
    'employee_id': [101, 102, 103],
    'name': ['Bob', 'Alice', 'Frank']
}
managers_df = pd.DataFrame(managers_data)

# Employees DataFrame
employees_data = {
    'employee_id': [201, 202, 203, 204],
    'name': ['Carol', 'Dave', 'Erin', 'James'],
    'reports_to': [101, 101, 102, 103],
    'revenue': [62000, 58000, 55000, 48000]
}
employees_df = pd.DataFrame(employees_data)

# Display the dataframes
print("Managers DataFrame:")
print(managers_df)
print("\nEmployees DataFrame:")
print(employees_df)

# 2. Import dataframes to SQL
# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Write DataFrames to SQL
managers_df.to_sql('Managers', engine, index=False, if_exists='replace')
employees_df.to_sql('Employees', engine, index=False, if_exists='replace')

# 3. SQL code to generate the Merged table
query = """
SELECT 
    m.name AS "Manager Name",
    e.name AS "Employee Name",
    e.revenue AS "Revenue"
FROM 
    Managers m 
JOIN 
    Employees e 
ON 
    m.employee_id = e.reports_to
ORDER BY
    m.name, e.name
"""

# 4. Execute the query and print the Merged table
print("\nMerged Table (SQL Result):")
merged_df = pd.read_sql_query(query, engine)
print(merged_df)


Managers DataFrame:
   employee_id   name
0          101    Bob
1          102  Alice
2          103  Frank

Employees DataFrame:
   employee_id   name  reports_to  revenue
0          201  Carol         101    62000
1          202   Dave         101    58000
2          203   Erin         102    55000
3          204  James         103    48000

Merged Table (SQL Result):
  Manager Name Employee Name  Revenue
0        Alice          Erin    55000
1          Bob         Carol    62000
2          Bob          Dave    58000
3        Frank         James    48000


Let's Break down exactly what is happening in the Query above:


```sql
SELECT 
    m.name AS "Manager Name",
    e.name AS "Employee Name",
    e.revenue AS "Revenue"
FROM 
    Managers m 
JOIN 
    Employees e 
ON 
    m.employee_id = e.reports_to
ORDER BY
    m.name, e.name
```

Let's start from the inside out:
- The first operation being performed is "FROM". This is telling us which Table we will be using as our "main" table - which in SQL is usually referred to as the Left Table. It is useful to think of any JOIN operation as acting "to the right" of this table.
    - Notice that we include a "m" right next to the name of the table we are importing - this is what we call an alias. While this is not necessary for a simple JOIN statement like this, it help keep the query clean and organized. And, as we will see later on, it is necessary for more advanced JOIN statements (such as self-joins).
- The second operation being performed is "JOIN". This is essentially telling SQL to consider a table that is a combination of the Left and Right tables.
    - One very important aspect is that we must tell SQL how to merge these two tables - which is what we do by using the ON statement.
    - In more advanced queries, ON statements can use very complex conditions, including compound ones using logical operators such as AND/OR/NOT, but for now we are just saying "I want a row that combines the information from the two tables whenever the column "employeed_id" on the Manager table matches the "reports_to" column on the Employees table.
    - Notice that we had to specify which table each column came from here - and we can already see why aliases can come in handy.
    - A very important thing to notice is that, if there are multiple matches to the condition on the ON statement, any rows that satisfy it will be joined. This is what happened in our example - there are two rows in the Merged table that have Bob as the Manager Name, since he had two employees that reported to him.
    - Another important thing to know is that, in a basic JOIN statement, any rows that go unmatched are not included in the final result. We will see how more advanced JOIN statements - such as LEFT JOIN - allow us to bypass this.
- We then go to our SELECT statement. As before, this is just telling SQL which information we actually want it to retrieve for us. But, since the information now might come from more than one source, we again need to specify from which table each column is coming from.
- The rest of the statements is similar to a basic Query - we can filter, order, limit/offset, etc - just like we did in the previous workshop


![JOIN Query Flow](sql-query-flow.svg)
---

In [11]:
# IDEA FOR CHALLENGE 1: Produce a table which tells us the employee name, their department, and their total salary - in this order


# Create Departments dataframe
departments_data = {
    'department': ['Engineering', 'Marketing', 'Sales', 'HR', 'Engineering'],
    'employee_name': ['John Smith', 'Sarah Johnson', 'Michael Lee', 'John Smith', 'Emma Davis'],
    'employee_id': [101, 102, 103, 104, 105]
}
departments_df = pd.DataFrame(departments_data)

# Create Salaries dataframe
salaries_data = {
    'employee_name': ['John Smith', 'Sarah Johnson', 'Michael Lee', 'John Smith', 'David Wilson'],
    'employee_id': [101, 102, 103, 104, 106],
    'hours_worked': [40, 35, 38, 30, 25],
    'hourly_salary': [35.50, 42.00, 38.75, 45.25, 33.00]
}
salaries_df = pd.DataFrame(salaries_data)

print("Departments DataFrame:")
print(departments_df)
print("\nSalaries DataFrame:")
print(salaries_df)

# Note the key aspects:
# - Two employees named "John Smith" (employee_id 101 and 104)
# - "Emma Davis" (id 105) is in Departments but not in Salaries
# - "David Wilson" (id 106) is in Salaries but not in Departments

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Import the dataframes to SQL tables
departments_df.to_sql('Departments', engine, index=False)
salaries_df.to_sql('Salaries', engine, index=False)

# Different types of joins

# Inner join (only matching records)
inner_join_query = """
SELECT
    d.employee_name,
    d.department,
    s.hours_worked * s.hourly_salary AS total_salary
FROM 
    Departments d
INNER JOIN 
    Salaries s
ON 
    d.employee_id = s.employee_id
ORDER BY
    d.department, d.employee_id
"""

print("\n JOIN Results:")
inner_join_results = pd.read_sql_query(inner_join_query, engine)
print(inner_join_results)

Departments DataFrame:
    department  employee_name  employee_id
0  Engineering     John Smith          101
1    Marketing  Sarah Johnson          102
2        Sales    Michael Lee          103
3           HR     John Smith          104
4  Engineering     Emma Davis          105

Salaries DataFrame:
   employee_name  employee_id  hours_worked  hourly_salary
0     John Smith          101            40          35.50
1  Sarah Johnson          102            35          42.00
2    Michael Lee          103            38          38.75
3     John Smith          104            30          45.25
4   David Wilson          106            25          33.00

 JOIN Results:
   employee_name   department  total_salary
0     John Smith  Engineering        1420.0
1     John Smith           HR        1357.5
2  Sarah Johnson    Marketing        1470.0
3    Michael Lee        Sales        1472.5


⚠️ **Warning:** Always make sure that your joining condition is present and accurate. Otherwise SQL will perform what we call a Cartesian Product - making every possible combination between the rows - which can become huge really fast.

## 2.1 `ON` versus `USING`
While the ON clause is extremely flexible, if the two tables share a column, we can use the simpler USING() method.

⚠️  **Warning:** When using "USING" to JOIN tables, always make sure that the names are identical in both tables.

In [14]:
using_join_query = """
SELECT
    d.employee_name,
    d.department,
    s.hours_worked * s.hourly_salary AS total_salary
FROM 
    Departments d
INNER JOIN 
    Salaries s
USING(employee_id)
"""

print("\n USING() JOIN Results:")
using_join_results = pd.read_sql_query(using_join_query, engine)
print(using_join_results)


 USING() JOIN Results:
   employee_name   department  total_salary
0     John Smith  Engineering        1420.0
1  Sarah Johnson    Marketing        1470.0
2    Michael Lee        Sales        1472.5
3     John Smith           HR        1357.5


But is important to be careful - using a non-primary key column can lead to issues

In [16]:
using_join_query_2 = """
SELECT
    d.employee_name,
    d.department,
    s.hours_worked * s.hourly_salary AS total_salary
FROM 
    Departments d
INNER JOIN 
    Salaries s
USING(employee_name)
"""

print("\n Problematic USING() JOIN Results:")
using_join_results_2 = pd.read_sql_query(using_join_query_2, engine)
print(using_join_results_2)


 Problematic USING() JOIN Results:
   employee_name   department  total_salary
0     John Smith  Engineering        1357.5
1     John Smith  Engineering        1420.0
2  Sarah Johnson    Marketing        1470.0
3    Michael Lee        Sales        1472.5
4     John Smith           HR        1357.5
5     John Smith           HR        1420.0


🔔 **Question:** Can anyone figure out what went wrong here?

## 2.2 Multiple Joins
We don't have to stop at two - we can use multiple joins at once. And what is interesting is that we can get pretty creative with the ON conditions.

In [72]:
# Create the four tables
# 1. Employees: employee_id and names
employees_data = {
    'employee_id': [101, 102, 103, 104, 105, 106],
    'name': ['John Smith', 'Sarah Johnson', 'Michael Lee', 'Emma Davis', 'David Wilson', 'Jennifer Lopez']
}
employees_df = pd.DataFrame(employees_data)

# 2. Employee Ranks: employee_id and their job rank
ranks_data = {
    'employee_id': [101, 102, 103, 104, 105, 107],  # Note: 107 is not in employees, 106 is missing
    'rank': ['Senior', 'Manager', 'Junior', 'Senior', 'Junior', 'Manager']
}
ranks_df = pd.DataFrame(ranks_data)

# 3. Wages: job rank and hourly wage
wages_data = {
    'rank': ['Junior', 'Senior', 'Manager', 'Director'],  # Note: No employees with "Director" rank
    'hourly_wage': [25.50, 35.75, 45.00, 65.00]
}
wages_df = pd.DataFrame(wages_data)

# 4. Hours: employee_id and hours worked
hours_data = {
    'employee_id': [101, 102, 103, 105, 106, 108],  # Note: 108 is not in employees, 104 is missing
    'hours_worked': [40, 45, 38, 35, 42, 30]
}
hours_df = pd.DataFrame(hours_data)

# Display the dataframes
print("Employees Table:")
print(employees_df)
print("\nRanks Table:")
print(ranks_df)
print("\nWages Table:")
print(wages_df)
print("\nHours Table:")
print(hours_df)

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Import the dataframes to SQL tables
employees_df.to_sql('Employees', engine, index=False)
ranks_df.to_sql('Ranks', engine, index=False)
wages_df.to_sql('Wages', engine, index=False)
hours_df.to_sql('Hours', engine, index=False)

# JOIN
inner_join_query = """
SELECT 
    e.name,
    w.hourly_wage * h.hours_worked AS total_pay
FROM 
    Employees e
INNER JOIN 
    Ranks r ON e.employee_id = r.employee_id
INNER JOIN 
    Wages w ON r.rank = w.rank
INNER JOIN 
    Hours h ON e.employee_id = h.employee_id
ORDER BY
    total_pay DESC
"""

print("\n Join Results:")
inner_join_results = pd.read_sql_query(inner_join_query, engine)
print(inner_join_results)

Employees Table:
   employee_id            name
0          101      John Smith
1          102   Sarah Johnson
2          103     Michael Lee
3          104      Emma Davis
4          105    David Wilson
5          106  Jennifer Lopez

Ranks Table:
   employee_id     rank
0          101   Senior
1          102  Manager
2          103   Junior
3          104   Senior
4          105   Junior
5          107  Manager

Wages Table:
       rank  hourly_wage
0    Junior        25.50
1    Senior        35.75
2   Manager        45.00
3  Director        65.00

Hours Table:
   employee_id  hours_worked
0          101            40
1          102            45
2          103            38
3          105            35
4          106            42
5          108            30

 Join Results:
            name  total_pay
0  Sarah Johnson     2025.0
1     John Smith     1430.0
2    Michael Lee      969.0
3   David Wilson      892.5


🔔 **Question:** Why can't we see the total_pay for Jennifer Lopez and Emma Davis?

This is an extremely important property of the JOIN command - it is what we call an INNER JOIN.

In practice, this means that we only return rows that are present on both tables being merged. So, in the previous example, sinced Jennifer did not have an entry in the Rank table, this entry was dropped. Similarly, since Emma did not have her hours recorded on the Hours table, her entry was dropped on the last JOIN.

Of course, there are many situations that this is not what we want. For example, we might want to return NULL for Jennifer Lopez, to indicate that there was a missing record, and 0 for Emma Davis, since she did not work any hours. 

In this workshop we will be focusing on the two most common advanced methods of JOIN - LEFT JOIN and SELF JOIN - but for those curious, there are others, such as RIGHT JOIN, FULL JOIN AND CROSS JOIN.

## 2.3 Advanced Join Variants
- **LEFT JOIN**: retains all rows from the left table.
    - This is why it is important to distinguish which table is being used in the FROM statement, and which is being brought by the JOIN statement
    - Specially useful to handle missing data 
- **SELF JOIN**: the table is joined to itself
    - We essentially deal with two tables - one of them being a duplicate of the first - and then JOIN them
    - Very useful as a filtering tool

🔔 **Question:** Can anyone think of an example in which we might want to join a table with itself?

In [78]:
# Create the four tables
# 1. Employees: employee_id and names
employees_data = {
    'employee_id': [101, 102, 103, 104, 105, 106],
    'name': ['John Smith', 'Sarah Johnson', 'Michael Lee', 'Emma Davis', 'David Wilson', 'Jennifer Lopez']
}
employees_df = pd.DataFrame(employees_data)

# 2. Employee Ranks: employee_id and their job rank
ranks_data = {
    'employee_id': [101, 102, 103, 104, 105, 107],  # Note: 107 is not in employees, 106 is missing
    'rank': ['Senior', 'Manager', 'Junior', 'Senior', 'Junior', 'Manager']
}
ranks_df = pd.DataFrame(ranks_data)

# Display the dataframes
print("Employees Table:")
print(employees_df)
print("\nRanks Table:")
print(ranks_df)

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Import the dataframes to SQL tables
employees_df.to_sql('Employees', engine, index=False)
ranks_df.to_sql('Ranks', engine, index=False)
wages_df.to_sql('Wages', engine, index=False)
hours_df.to_sql('Hours', engine, index=False)

# LEFT JOIN
left_join_query = """
SELECT *
FROM 
    Employees e
LEFT JOIN 
    Ranks r ON e.employee_id = r.employee_id
"""

print("\nLeft Join Results:")
left_join_results = pd.read_sql_query(left_join_query, engine)
print(left_join_results)

Employees Table:
   employee_id            name
0          101      John Smith
1          102   Sarah Johnson
2          103     Michael Lee
3          104      Emma Davis
4          105    David Wilson
5          106  Jennifer Lopez

Ranks Table:
   employee_id     rank
0          101   Senior
1          102  Manager
2          103   Junior
3          104   Senior
4          105   Junior
5          107  Manager

Left Join Results:
   employee_id            name  employee_id     rank
0          101      John Smith        101.0   Senior
1          102   Sarah Johnson        102.0  Manager
2          103     Michael Lee        103.0   Junior
3          104      Emma Davis        104.0   Senior
4          105    David Wilson        105.0   Junior
5          106  Jennifer Lopez          NaN     None


IDEA FOR CHALLENGE: Let's check that the order is indeed relevant. Let's use a LEFT JOIN between these two table - but now let's use the Rank table as the left one, and the Employees as the right one. What happened? Does that make sense?

One problem with using LEFT JOIN is that it often returns NONE values - but we might want to interpret them in a different way. For example, we saw that Emma Davis did not work any hours, but if we did a left join on Employees and Hours our result would indicate that she worked NONE hours. 

Luckily, SQL includes a function that allows us to taylor the behavior of NONE entries for a given query - the COALESCE command. The first argument of the function tells us which column to analyze, and the second entry what to replace NONE values by. Let's take a look at an example in practice

In [80]:
# Create Employees dataframe
employees_data = {
    'employee_id': [101, 102, 103, 104, 105, 106],
    'name': ['John Smith', 'Sarah Johnson', 'Michael Lee', 'Emma Davis', 'David Wilson', 'Jennifer Lopez']
}
employees_df = pd.DataFrame(employees_data)

# Create Hours dataframe (with some employees missing)
hours_data = {
    'employee_id': [101, 102, 103, 105, 106, 108],  # Note: 108 is not in employees, 104 is missing
    'hours_worked': [40, 45, 38, 35, 42, 30]
}
hours_df = pd.DataFrame(hours_data)

# Display the original dataframes
print("Employees Table:")
print(employees_df)
print("\nHours Table:")
print(hours_df)

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Import the dataframes to SQL tables
employees_df.to_sql('Employees', engine, index=False)
hours_df.to_sql('Hours', engine, index=False)

# LEFT JOIN query with COALESCE to handle NULL hours
left_join_query = """
SELECT 
    e.employee_id,
    e.name,
    COALESCE(h.hours_worked, 0) AS hours_worked
FROM 
    Employees e
LEFT JOIN 
    Hours h ON e.employee_id = h.employee_id
ORDER BY
    e.employee_id
"""

# Execute the query
left_join_results = pd.read_sql_query(left_join_query, engine)

# Display the results
print("\nLEFT JOIN Results with COALESCE:")
print(left_join_results)

Employees Table:
   employee_id            name
0          101      John Smith
1          102   Sarah Johnson
2          103     Michael Lee
3          104      Emma Davis
4          105    David Wilson
5          106  Jennifer Lopez

Hours Table:
   employee_id  hours_worked
0          101            40
1          102            45
2          103            38
3          105            35
4          106            42
5          108            30

LEFT JOIN Results with COALESCE:
   employee_id            name  hours_worked
0          101      John Smith            40
1          102   Sarah Johnson            45
2          103     Michael Lee            38
3          104      Emma Davis             0
4          105    David Wilson            35
5          106  Jennifer Lopez            42


Now let's talk a bit about SELF JOIN. As we mentioned before, the idea here is to join a table with itself. This is one of the cases in which using aliases is extremely important, since, by construction, both of the tables will have all columns in common!

Let's see an example on how this might work in practice.

In [84]:
# Create Employees dataframe with salary and reporting structure
employees_data = {
    'employee_id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'name': ['John', 'Sarah', 'Michael', 'Emma', 'David', 'Jennifer', 'Robert', 'Lisa', 'Kevin'],
    'salary': [150000, 120000, 140000, 115000, 125000, 95000, 160000, 90000, 80000],
    'reports_to': [None, 1, 1, 2, 2, 3, 3, 4, 5]  # None for CEO
}
employees_df = pd.DataFrame(employees_data)

# Display the original dataframe
print("Employees Table:")
print(employees_df)

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Import the dataframe to SQL table
employees_df.to_sql('Employees', engine, index=False)

# Self JOIN query to compare the salaries of employees and their managers
self_join_query = """
SELECT 
    e.name AS employee_name,
    e.salary AS employee_salary,
    m.name AS manager_name,
    m.salary AS manager_salary
FROM 
    Employees e
JOIN 
    Employees m ON e.reports_to = m.employee_id
"""

# Execute the query
results = pd.read_sql_query(self_join_query, engine)

# Display the results
print(results)


Employees Table:
   employee_id      name  salary  reports_to
0            1      John  150000         NaN
1            2     Sarah  120000         1.0
2            3   Michael  140000         1.0
3            4      Emma  115000         2.0
4            5     David  125000         2.0
5            6  Jennifer   95000         3.0
6            7    Robert  160000         3.0
7            8      Lisa   90000         4.0
8            9     Kevin   80000         5.0
  employee_name  employee_salary manager_name  manager_salary
0         Sarah           120000         John          150000
1       Michael           140000         John          150000
2          Emma           115000        Sarah          120000
3         David           125000        Sarah          120000
4      Jennifer            95000      Michael          140000
5        Robert           160000      Michael          140000
6          Lisa            90000         Emma          115000
7         Kevin            80000     

There are a few things worth noticing here:
- When we selected the columns from each table, it was important to rename them - since they had the same original names!
- Notice that John did not report to anyone - so he was not included as an employee in the merged table. This could be adapted by using a LEFT JOIN
- Some managers appear many times, since more than one employee reports to them.



IDEA FOR CHALLENGE: SELF JOINs can be extremely useful as a filtering tool. A classic example is to return the name of the employees who earn more than their managers. Can you modify the previous query and return **only** the name of the employees who earn more than their managers? 


Of course, SELF JOINs can be used for purposes other than filtering. Another classic application is to find pairs that satisfy some criterion

In [31]:
# Create a dataframe with people, their roles, and valuations
valuations_data = {
    'person_name': ['John Smith', 'Sarah Johnson', 'Michael Lee', 'Emma Davis', 
                    'David Wilson', 'Jennifer Lopez', 'Robert Brown', 'Lisa Chen'],
    'role': ['buyer', 'seller', 'buyer', 'seller', 'buyer', 'seller', 'buyer', 'seller'],
    'value': [500, 400, 350, 300, 600, 550, 450, 500]
}
valuations_df = pd.DataFrame(valuations_data)

# Display the original dataframe
print("Valuations Table:")
print(valuations_df)

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Import the dataframe to SQL table
valuations_df.to_sql('Valuations', engine, index=False)

# Self JOIN query to find buyer-seller pairs where buyer values object more than seller
self_join_query = """
SELECT 
    b.person_name AS buyer_name,
    s.person_name AS seller_name
FROM 
    Valuations b
JOIN 
    Valuations s ON b.role = 'buyer' AND s.role = 'seller'
WHERE 
    b.value > s.value
ORDER BY
    b.person_name, s.person_name
"""

# Execute the query and display results
results = pd.read_sql_query(self_join_query, engine)
print("\nPairs where buyer values object more than seller:")
print(results)

Valuations Table:
      person_name    role  value
0      John Smith   buyer    500
1   Sarah Johnson  seller    400
2     Michael Lee   buyer    350
3      Emma Davis  seller    300
4    David Wilson   buyer    600
5  Jennifer Lopez  seller    550
6    Robert Brown   buyer    450
7       Lisa Chen  seller    500

Pairs where buyer values object more than seller:
     buyer_name     seller_name
0  David Wilson      Emma Davis
1  David Wilson  Jennifer Lopez
2  David Wilson       Lisa Chen
3  David Wilson   Sarah Johnson
4    John Smith      Emma Davis
5    John Smith   Sarah Johnson
6   Michael Lee      Emma Davis
7  Robert Brown      Emma Davis
8  Robert Brown   Sarah Johnson


<a id='subqueries'></a>
## 3 · Subqueries 

**Definition.** As the name says, a subquery is a query contained in another query. This allows us to perform auxiliary queries, and then use the results of these queries in our main query. Unlike the main query, subqueries are temporary - they only exist while the instance of the query is being worked out. 

**Purpose.** Subqueries are very useful when one needs to break down a complex question into multiple manageable individual parts.For example, one might want to summarize or filter a given table, and use the summarized/filtered results as the input of another query. 

**Common Use Case.** Getting summary statistics for each individual, while keeping information unrelated from the variable we are using to aggregate.

In [87]:
# Create sample data
orders_data = {
    'order_id': [1001, 1002, 1003, 1004, 1005],
    'customer_id': [101, 101, 102, 103, 103],
    'product': ['Laptop', 'Mouse', 'Monitor', 'Printer', 'Tablet'],
    'amount': [1200, 25, 350, 180, 450]
}
orders_df = pd.DataFrame(orders_data)

customers_data = {
    'customer_id': [101, 102, 103, 104],
    'customer_name': ['Alice', 'Bob', 'Charlie', 'David'],
    'customer_city': ['New York', 'Chicago', 'Los Angeles', 'Boston']
}
customers_df = pd.DataFrame(customers_data)

# Create database
engine = create_engine('sqlite:///:memory:')
orders_df.to_sql('Orders', engine, index=False)
customers_df.to_sql('Customers', engine, index=False)

without_subquery = """
SELECT customer_id, SUM(amount) as total_spent 
     FROM Orders 
     GROUP BY customer_id
ORDER BY
    total_spent DESC
"""

result = pd.read_sql_query(without_subquery, engine)
print("Customer totals withOUT all customer information:")
print(result)

# JOIN with a subquery to get customer totals with their information
join_on_subquery = """
SELECT 
    c.customer_id,
    c.customer_name, 
    c.customer_city,
    s.total_spent
FROM 
    Customers c
JOIN 
    (SELECT customer_id, SUM(amount) as total_spent 
     FROM Orders 
     GROUP BY customer_id) s
ON 
    c.customer_id = s.customer_id
ORDER BY
    total_spent DESC
"""

result = pd.read_sql_query(join_on_subquery, engine)
print("Customer totals with all customer information:")
print(result)

Customer totals withOUT all customer information:
   customer_id  total_spent
0          101         1225
1          103          630
2          102          350
Customer totals with all customer information:
   customer_id customer_name customer_city  total_spent
0          101         Alice      New York         1225
1          103       Charlie   Los Angeles          630
2          102           Bob       Chicago          350


There are two main ways of using subqueries:

1) To Filter results.

A classic example is to find all customers who spend more than average. 

In [35]:
# Create sample data
orders_data = {
    'order_id': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
    'customer_id': [101, 101, 102, 101, 103, 102, 104],
    'product': ['Laptop', 'Mouse', 'Monitor', 'Keyboard', 'Printer', 'Headphones', 'Tablet'],
    'amount': [1200, 25, 350, 45, 180, 95, 450]
}
orders_df = pd.DataFrame(orders_data)

customers_data = {
    'customer_id': [101, 102, 103, 104],
    'customer_name': ['Alice', 'Bob', 'Charlie', 'David']
}
customers_df = pd.DataFrame(customers_data)

# Create database
engine = create_engine('sqlite:///:memory:')
orders_df.to_sql('Orders', engine, index=False)
customers_df.to_sql('Customers', engine, index=False)

# Query to find customers who spend more than average
above_average_query = """
SELECT 
    c.customer_id,
    c.customer_name,
    SUM(o.amount) as total_spent,
    (SELECT AVG(total) FROM (
        SELECT customer_id, SUM(amount) as total
        FROM Orders
        GROUP BY customer_id
    )) as average_customer_spend
FROM 
    Customers c
JOIN 
    Orders o ON c.customer_id = o.customer_id
GROUP BY 
    c.customer_id, c.customer_name
HAVING 
    SUM(o.amount) > (
        SELECT AVG(total) FROM (
            SELECT customer_id, SUM(amount) as total
            FROM Orders
            GROUP BY customer_id
        )
    )
ORDER BY
    total_spent DESC
"""

result = pd.read_sql_query(above_average_query, engine)
print("Customers who spend more than the average:")
print(result)

Customers who spend more than the average:
   customer_id customer_name  total_spent  average_customer_spend
0          101         Alice         1270                  586.25


2) As a derived table to query from:

In [37]:
# Create sample employee data
employees_data = {
    'employee_id': range(1, 21),
    'employee_name': [f"Employee_{i}" for i in range(1, 21)],
    'department_id': [1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5],
    'salary': [45000, 52000, 50000, 60000, 61000, 59000, 75000, 70000, 72000, 71000, 
               90000, 88000, 95000, 91000, 89000, 42000, 44000, 41000, 43000, 45000]
}
employees_df = pd.DataFrame(employees_data)

# Create departments data
departments_data = {
    'department_id': range(1, 6),
    'department_name': ['Engineering', 'Marketing', 'Finance', 'HR', 'Operations']
}
departments_df = pd.DataFrame(departments_data)

# Display data
print("Employees Data:")
print(employees_df.head())

print("\nDepartments Data:")
print(departments_df)

# Create database
engine = create_engine('sqlite:///:memory:')
employees_df.to_sql('employees', engine, index=False)
departments_df.to_sql('departments', engine, index=False)

# Query using a FROM clause subquery
from_subquery = """
SELECT 
    d.department_name,
    ds.avg_salary
FROM 
    departments d
JOIN 
    (SELECT 
        department_id,
        AVG(salary) AS avg_salary
     FROM 
        employees
     GROUP BY 
        department_id) AS ds
ON 
    d.department_id = ds.department_id
ORDER BY 
    ds.avg_salary DESC
"""

# Execute the query
results = pd.read_sql_query(from_subquery, engine)

# Display the results
print("\nDepartment Average Salaries (using FROM subquery):")
print(results)

Employees Data:
   employee_id employee_name  department_id  salary
0            1    Employee_1              1   45000
1            2    Employee_2              1   52000
2            3    Employee_3              1   50000
3            4    Employee_4              2   60000
4            5    Employee_5              2   61000

Departments Data:
   department_id department_name
0              1     Engineering
1              2       Marketing
2              3         Finance
3              4              HR
4              5      Operations

Department Average Salaries (using FROM subquery):
  department_name  avg_salary
0              HR     90600.0
1         Finance     72000.0
2       Marketing     60000.0
3     Engineering     49000.0
4      Operations     43000.0


As promised in the first workshop, we can also use subqueries in combination with IN to check for membership against entire tables - usually the result of a subquery.

For example, let's say that we want to select the names of all customers who have purchased on Electronic item, but the information between consumers, products and purchases are all on separate tables. We could first JOIN them, then use a WHERE statement to filter them. But this would create a very large merged table. Instead, we can just use an IN statement with a subquery:

In [89]:
# Create a database with customers and their purchases
customers_data = {
    'customer_id': [101, 102, 103, 104, 105],
    'customer_name': ['Alice', 'Bob', 'Charlie', 'David', 'Emma']
}

products_data = {
    'product_id': [1, 2, 3, 4, 5, 6, 7],
    'product_name': ['Laptop', 'Headphones', 'Coffee Maker', 'Blender', 'Smartphone', 'Desk Chair', 'Bookshelf'],
    'category': ['Electronics', 'Electronics', 'Appliances', 'Appliances', 'Electronics', 'Furniture', 'Furniture'],
    'price': [899.99, 149.99, 89.99, 59.99, 699.99, 199.99, 129.99]
}

purchases_data = {
    'purchase_id': [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009],
    'customer_id': [101, 101, 102, 103, 103, 104, 104, 105, 105],
    'product_id': [1, 5, 2, 3, 6, 4, 7, 5, 6],
    'purchase_date': ['2023-01-15', '2023-02-10', '2023-01-22', '2023-02-05', 
                      '2023-03-12', '2023-01-30', '2023-02-28', '2023-03-03', '2023-03-18']
}

# Create dataframes
customers_df = pd.DataFrame(customers_data)
products_df = pd.DataFrame(products_data)
purchases_df = pd.DataFrame(purchases_data)

# Create database
engine = create_engine('sqlite:///:memory:')
customers_df.to_sql('customers', engine, index=False)
products_df.to_sql('products', engine, index=False)
purchases_df.to_sql('purchases', engine, index=False)

# Display tables
print("Customers:")
print(customers_df)
print("\nProducts:")
print(products_df)
print("\nPurchases:")
print(purchases_df)

# Example 1: Using IN with a subquery to find customers who purchased Electronics
electronics_query = """
SELECT  
    c.customer_name
FROM 
    customers c
WHERE 
    c.customer_id IN (
        SELECT DISTINCT p.customer_id
        FROM purchases p
        JOIN products pr ON p.product_id = pr.product_id
        WHERE pr.category = 'Electronics'
    )
ORDER BY
    c.customer_id
"""

print("\nCustomers who purchased Electronics:")
print(pd.read_sql_query(electronics_query, engine))

Customers:
   customer_id customer_name
0          101         Alice
1          102           Bob
2          103       Charlie
3          104         David
4          105          Emma

Products:
   product_id  product_name     category   price
0           1        Laptop  Electronics  899.99
1           2    Headphones  Electronics  149.99
2           3  Coffee Maker   Appliances   89.99
3           4       Blender   Appliances   59.99
4           5    Smartphone  Electronics  699.99
5           6    Desk Chair    Furniture  199.99
6           7     Bookshelf    Furniture  129.99

Purchases:
   purchase_id  customer_id  product_id purchase_date
0         1001          101           1    2023-01-15
1         1002          101           5    2023-02-10
2         1003          102           2    2023-01-22
3         1004          103           3    2023-02-05
4         1005          103           6    2023-03-12
5         1006          104           4    2023-01-30
6         1007        

<a id='ctes'></a>
## 4 · Common Table Expressions (CTEs) 

Subqueries are a great way of breaking down a complex query into smaller, more manageable subparts. But very often these subqueries can become quite long, and given that one must include the full query inside another query, they can become very hard to read. 

CTEs are a way of solving this issue. Instead of rewriting the entire subquery inside the main query, we first give aliases to our subqueries, and then refer to them in the main query. 


Basic Syntax   

```sql

WITH cte_name AS (
    SELECT column1, column2, ...
    FROM table
    WHERE condition)
SELECT * 
FROM cte_name;
Key Components
```

WITH clause: Introduces one or more CTEs
cte_name: Gives a name to the temporary result set
Main query: References the CTE like a regular table

🔔 **Question:** Would you say that CTEs improve readability? Can you think of an example in which the code is easier to read using subqueries instead?

Multiple CTEs

```sql

WITH cte1 AS (
    SELECT column1 FROM table1
),
cte2 AS (
    SELECT column2 FROM table2
)
SELECT *
FROM cte1
JOIN cte2 ON cte1.column = cte2.column;
```

💡 **Tip:** CTE's can be referenced multiple times in the same query, which improve readability and prevents errors!

In [101]:
# Create sample data
orders_data = {
    'order_id': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
    'customer_id': [101, 101, 102, 101, 103, 102, 104],
    'product': ['Laptop', 'Mouse', 'Monitor', 'Keyboard', 'Printer', 'Headphones', 'Tablet'],
    'amount': [1200, 25, 350, 45, 180, 95, 450]
}
orders_df = pd.DataFrame(orders_data)

customers_data = {
    'customer_id': [101, 102, 103, 104],
    'customer_name': ['Alice', 'Bob', 'Charlie', 'David']
}
customers_df = pd.DataFrame(customers_data)

# Create database
engine = create_engine('sqlite:///:memory:')
orders_df.to_sql('Orders', engine, index=False)
customers_df.to_sql('Customers', engine, index=False)

# Query using CTEs 
cte_query = """
WITH customer_totals AS (
    SELECT 
        customer_id, 
        SUM(amount) as total_spent
    FROM 
        Orders
    GROUP BY 
        customer_id
),
avg_spending AS (
    SELECT 
        AVG(total_spent) as avg_spend
    FROM 
        customer_totals
)
SELECT 
    c.customer_id,
    c.customer_name,
    ct.total_spent,
    (SELECT avg_spend FROM avg_spending) as average_customer_spend
FROM 
    Customers c
JOIN 
    customer_totals ct ON c.customer_id = ct.customer_id
WHERE 
    ct.total_spent > (SELECT avg_spend FROM avg_spending)
ORDER BY
    ct.total_spent DESC
"""

result = pd.read_sql_query(cte_query, engine)
print("Customers who spend more than the average:")
print(result)

Customers who spend more than the average:
   customer_id customer_name  total_spent  average_customer_spend
0          101         Alice         1270                  586.25


⚠️ **Warning:** We cannot reference CTEs directly in a query - this is what we call (lack of) "portability".

```sql
WITH average_data AS (
    SELECT AVG(value) AS avg_value FROM table
)
SELECT *
FROM other_table
WHERE value > average_data.avg_value 
```
Whenever we want to reference a CTE, we must either use a subquery:

```sql
WHERE value > (SELECT avg_value FROM average_data)
```

Or JOIN with the CTE:

```sql
JOIN average_data ON 1=1
WHERE value > average_data.avg_value
```

💡 **Tip:** In this last example, we used a common trick: choose an expression that is always true to add a constant column to the table.

<a id='pivot'></a>
## 5 · Pivoting and Unpivoting (Or Melting) 

Pivoting is the process of turning rows into columns, Unpivoting (also called Melting) is the inverse process. 

A common application is when we would like to group information that applies to the same individual. For example, we might have a list of all the different transactions that different clients made, including amount and date. But maybe we would like to understand how purchases vary across the days of the week for each given customer. So we can turn a table from having many rows and three columns, to having one row for each consumer, and 8 columns: one representing the customer name (or any other identification), and one with the transaction amount for that customer on each day of the week. 

This is done by using the UNION (or UNION ALL) functions - which are analogues to JOINS, but instead of putting different columns side by side, they combine results from different queries on top of each other. 

The main difference between UNION and UNION ALL is that the former removes duplicate rows, which increases the computational cost of the function.

Two additional observations:
- The column names will be taken from the first one selected
- All SELECT statements must have the same number of columns, and corresponding columns must have the same data types.

In [46]:
# Create sample data - expenditure by person and day
expenditures_data = {
    'person': ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie', 'Charlie', 'Alice'],
    'expenditure': [100, 150, 200, 250, 175, 125, 90],
    'day_of_purchase': ['Monday', 'Tuesday', 'Monday', 'Wednesday', 'Tuesday', 'Friday', 'Friday']
}
expenditures_df = pd.DataFrame(expenditures_data)

print("Original Data:")
print(expenditures_df)

# Create database
engine = create_engine('sqlite:///:memory:')
expenditures_df.to_sql('Expenditures', engine, index=False)

# Pivot using UNION ALL
pivot_query = """
SELECT person,
       SUM(CASE WHEN day_of_purchase = 'Monday' THEN expenditure ELSE 0 END) AS Monday,
       SUM(CASE WHEN day_of_purchase = 'Tuesday' THEN expenditure ELSE 0 END) AS Tuesday,
       SUM(CASE WHEN day_of_purchase = 'Wednesday' THEN expenditure ELSE 0 END) AS Wednesday,
       SUM(CASE WHEN day_of_purchase = 'Thursday' THEN expenditure ELSE 0 END) AS Thursday,
       SUM(CASE WHEN day_of_purchase = 'Friday' THEN expenditure ELSE 0 END) AS Friday
FROM Expenditures
GROUP BY person
"""

# Execute query
result = pd.read_sql_query(pivot_query, engine)
print("\nPivoted Data:")
print(result)

Original Data:
    person  expenditure day_of_purchase
0    Alice          100          Monday
1    Alice          150         Tuesday
2      Bob          200          Monday
3      Bob          250       Wednesday
4  Charlie          175         Tuesday
5  Charlie          125          Friday
6    Alice           90          Friday

Pivoted Data:
    person  Monday  Tuesday  Wednesday  Thursday  Friday
0    Alice     100      150          0         0      90
1      Bob     200        0        250         0       0
2  Charlie       0      175          0         0     125


Melting, the opposite process, is very useful when we want to do analysis regarding a variable that is not the one determining the rows. For example, say that we have sales data in which each row corresponds to a different product, and different columns represent different years. If we instead want to analyze the expenditure in different years, we can melt the table, and then use grouping or filtering to select a given year.

In [48]:
import pandas as pd
from sqlalchemy import create_engine

# Create concise wide-format data with three years
sales_data = {
    'product': ['Laptop', 'Phone', 'Tablet'],
    'sales_2021': [1200, 800, 350],
    'sales_2022': [1500, 950, 400],
    'sales_2023': [1800, 1100, 500]
}
sales_df = pd.DataFrame(sales_data)

# Create database
engine = create_engine('sqlite:///:memory:')
sales_df.to_sql('YearlySales', engine, index=False)

# Melt the data using UNION ALL
melt_query = """
SELECT product, '2021' AS year, sales_2021 AS sales FROM YearlySales
UNION ALL
SELECT product, '2022' AS year, sales_2022 AS sales FROM YearlySales
UNION ALL
SELECT product, '2023' AS year, sales_2023 AS sales FROM YearlySales
ORDER BY product, year
"""

# Execute and display results
result = pd.read_sql_query(melt_query, engine)
print("Original Wide Data:")
print(sales_df)
print("\nMelted Data:")
print(result)

Original Wide Data:
  product  sales_2021  sales_2022  sales_2023
0  Laptop        1200        1500        1800
1   Phone         800         950        1100
2  Tablet         350         400         500

Melted Data:
  product  year  sales
0  Laptop  2021   1200
1  Laptop  2022   1500
2  Laptop  2023   1800
3   Phone  2021    800
4   Phone  2022    950
5   Phone  2023   1100
6  Tablet  2021    350
7  Tablet  2022    400
8  Tablet  2023    500


<a id='window'></a>
## 6 · Window Functions 
**Definition.** A window is a subset of the table that is related in some prespecified way, in a very similar way that GROUP BY operates - with the crucial difference that it does not collapse the different rows into one, which allows us to preserve information. 

Window Functions operate separately into different windows. Common examples are

- RANK - allow us to obtain the rank for different observations inside a given group. For example, we might want to see how much a consumer spent on his first purchase. So we would create a window for each consumer, apply the RANK window function, and then select those that have rank = 1.
- DENSE_RANK - Similar to rank, but RANK jumps ranks if there are ties, while DENSE_RANK always has rankings as consecutive numbers. You might have seen this happening in college rankings!
- ROW_NUMBER() - returns the row number of each observation. Very useful when the table lacks a primary key.
- SUM - when used as a window function, allow us to perform cumulative sums - for example, cumulative sales up to a given date by each salesperson.
- AVG/MAX/MIN - same as their aggregate versions, but allowing to keep information rather than collapsing rows.
- LAG/LEAD - very useful in the context of time series, allow us to look at the previous/next value of a series.

Basic Syntax: WINDOW_FUNCTION() OVER (PARTITION BY columns ORDER BY columns)

Exception: SUM(column) OVER (PARTITION BY columns)

In [105]:
import pandas as pd
from sqlalchemy import create_engine

# Create a versatile sales dataset
sales_data = {
    'salesperson': ['Alice', 'Bob', 'Charlie', 'David', 'Emma',
                   'Alice', 'Bob', 'Charlie', 'David', 'Emma',
                   'Alice', 'Bob', 'Charlie', 'David', 'Emma',
                   'Alice', 'Bob', 'Charlie', 'David', 'Emma'],
    'quarter': [1, 1, 1, 1, 1, 
               2, 2, 2, 2, 2, 
               3, 3, 3, 3, 3, 
               4, 4, 4, 4, 4],
    'sales': [120000, 120000, 85000, 95000, 95000,
             140000, 115000, 90000, 100000, 100000,
             160000, 125000, 95000, 110000, 110000,
             180000, 150000, 105000, 115000, 115000]
}
sales_df = pd.DataFrame(sales_data)

# Create database
engine = create_engine('sqlite:///:memory:')
sales_df.to_sql('SalesData', engine, index=False)

print("Dataset for all examples:")
print(sales_df.head(10))
print("...")

# Example 1: RANK - ranking salespeople by total annual sales
rank_query = """
SELECT 
    salesperson,
    SUM(sales) AS annual_sales,
    RANK() OVER(ORDER BY SUM(sales) DESC) AS sales_rank
FROM 
    SalesData
GROUP BY 
    salesperson
ORDER BY 
    sales_rank
"""
print("\n1. RANK Example - Annual Sales Rankings:")
print(pd.read_sql_query(rank_query, engine))

# Example 2: DENSE_RANK - ranking quarterly performance 
dense_rank_query = """
SELECT 
    quarter,
    salesperson,
    sales,
    DENSE_RANK() OVER(PARTITION BY quarter ORDER BY sales DESC) AS quarter_rank
FROM 
    SalesData
ORDER BY 
    quarter, quarter_rank
"""
print("\n2. DENSE_RANK Example - Quarterly Performance Rankings:")
print(pd.read_sql_query(dense_rank_query, engine))

# Example 3: ROW_NUMBER - identifying top performer each quarter
row_number_query = """
SELECT 
    quarter,
    salesperson,
    sales
FROM (
    SELECT 
        quarter,
        salesperson,
        sales,
        ROW_NUMBER() OVER(PARTITION BY quarter ORDER BY sales DESC) AS row_num
    FROM 
        SalesData
) ranked
WHERE 
    row_num = 1
ORDER BY 
    quarter
"""
print("\n3. ROW_NUMBER Example - Top Performer Each Quarter:")
print(pd.read_sql_query(row_number_query, engine))

# Example 4: SUM() OVER - running total of sales per person
sum_query = """
SELECT 
    salesperson,
    quarter,
    sales,
    SUM(sales) OVER(PARTITION BY salesperson ORDER BY quarter) AS running_total
FROM 
    SalesData
ORDER BY 
    salesperson, quarter
"""
print("\n4. SUM() OVER Example - Running Sales Totals:")
print(pd.read_sql_query(sum_query, engine))

# Example 5: LAG - quarter-over-quarter growth
lag_query = """
SELECT 
    salesperson,
    quarter,
    sales,
    LAG(sales, 1) OVER(PARTITION BY salesperson ORDER BY quarter) AS prev_quarter_sales,
    CASE 
        WHEN LAG(sales, 1) OVER(PARTITION BY salesperson ORDER BY quarter) IS NOT NULL
        THEN ROUND((sales - LAG(sales, 1) OVER(PARTITION BY salesperson ORDER BY quarter)) * 100.0 / 
                  LAG(sales, 1) OVER(PARTITION BY salesperson ORDER BY quarter), 1)
        ELSE NULL
    END AS growth_percent
FROM 
    SalesData
ORDER BY 
    salesperson, quarter
"""
print("\n5. LAG Example - Quarter-over-Quarter Growth:")
print(pd.read_sql_query(lag_query, engine))

Dataset for all examples:
  salesperson  quarter   sales
0       Alice        1  120000
1         Bob        1  120000
2     Charlie        1   85000
3       David        1   95000
4        Emma        1   95000
5       Alice        2  140000
6         Bob        2  115000
7     Charlie        2   90000
8       David        2  100000
9        Emma        2  100000
...

1. RANK Example - Annual Sales Rankings:
  salesperson  annual_sales  sales_rank
0       Alice        600000           1
1         Bob        510000           2
2        Emma        420000           3
3       David        420000           3
4     Charlie        375000           5

2. DENSE_RANK Example - Quarterly Performance Rankings:
    quarter salesperson   sales  quarter_rank
0         1       Alice  120000             1
1         1         Bob  120000             1
2         1       David   95000             2
3         1        Emma   95000             2
4         1     Charlie   85000             3
5         2   

<div class="alert alert-success">

## 7 · Key Points <a id='keypoints'></a>
- Joins combine tables using foreign keys.
- Subqueries allow us to break down complex queries into more manageable parts.
- CTEs make subqueries easier to read and re-use.
- Pivoting and Melting allow us to reshape the table to better fit our purposes.
- Window functions are an alternative to aggregate functions but still allow us to keep individual information.

</div>